In [1]:
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains

from time import sleep
import tqdm
import json
import time

import requests
from bs4 import BeautifulSoup
from html2text import html2text

import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
#Download driver using the link at the top of the notebook
chrome_driver_path = "chromedriver.exe"
links_filename = "links.txt"

In [7]:
categories = ["science", "sports", "entertainment", "lifestyle"]

In [12]:
def parse_category_foxnews(category, pages, output):
    prefs = {"profile.managed_default_content_settings.images": 2,
             "profile.default_content_settings.cookies": 2}
    chrome_options = Options()
    chrome_options.add_extension('adblock.crx')
    chrome_options.add_experimental_option("prefs",prefs)
    chrome_options.add_argument('--disable-application-cache')
    driver = Chrome(executable_path=chrome_driver_path, chrome_options=chrome_options)
    driver.implicitly_wait(10)
    driver.close()
    print("Current session is {}".format(driver.session_id))
    driver.switch_to.window(driver.window_handles[0])
    driver.get('chrome-extension://gighmmpiobklfepjocnamgkkbiglidom/options.html')
    driver.find_element('id', 'acceptable_ads').click() #adblock customization
    driver.get('https://www.foxnews.com/' + category)

    SCROLL_PAUSE_TIME = 2

    for i in range(pages):
        driver.find_element(By.CLASS_NAME, 'load-more').click()
        time.sleep(SCROLL_PAUSE_TIME)

    for block in tqdm.tqdm(driver.find_elements(By.CLASS_NAME,
    "main-content"),position=0):
        for element in tqdm.tqdm(block.find_elements(By.XPATH,
        "//article"),position=0):    
            with open(output, 'a') as file:
                if 'video' in element.find_element(By.TAG_NAME, 'a').get_attribute('href'):
                    continue
                file.write(json.dumps({
                    'link' : element.find_element(By.TAG_NAME, 'a').get_attribute('href'),
                    'category': category
                },ensure_ascii=False) + '\n') 
    driver.quit()


In [14]:
for cat in categories:
    parse_category_foxnews(cat, 100, links_filename)

Current session is e548d8d30f62e6d13a4ddc6363686591


100%|██████████| 1/1 [00:53<00:00, 53.86s/it]


Current session is a89179ec26a1adc6a0e72f47b50a2130


100%|██████████| 1/1 [00:58<00:00, 58.55s/it]


Current session is 6eef12e622543be935a93ddff8a78e32


100%|██████████| 1/1 [00:54<00:00, 54.95s/it]


Current session is b5fe5216b580d83823aec345dee31cbb


100%|██████████| 1/1 [00:40<00:00, 40.84s/it]


In [7]:
import json
from newspaper import Config
from newspaper import Article
from newspaper.utils import BeautifulSoup

USER_AGENT = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:78.0) Gecko/20100101 Firefox/78.0'

config = Config()
config.browser_user_agent = USER_AGENT
config.request_timeout = 10
links = []

jsondict = {
        'catalog' : [

        ]
    }

with open('links.txt', 'r') as f:
    for line in tqdm.tqdm(f.readlines()):
        dictionary = json.loads(line)
        links.append(dictionary['link'])
        #print(links[-1])
        base_url = links[-1]
        article = Article(base_url, config=config)
        article.download()
        article.parse()

        article_id = base_url
        article_title = article.title
        article_category = dictionary['category']

        article_meta_data = article.meta_data
        article_tags = article_meta_data['news_keywords']
        article_text = article.text

        jsondict['catalog'].append(
                        {
                            'article_id': article_id,
                            'title': article_title,
                            'category': article_category,
                            'tags': article_tags,
                            'text': article_text,
                        }
        )
    with open("news.json", "w") as write_file:
        json.dump(jsondict, write_file)
        

100%|██████████| 4077/4077 [31:11<00:00,  2.18it/s] 


In [ ]:
!pip install newspaper3k